In [0]:
import pandas as pd
from bsedata.bse import BSE
import time
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

github_excel_url = "https://raw.githubusercontent.com/jangid6/Stock-ETL-Project/main/Equity.xlsx"
engine = 'openpyxl' 
Equity = pd.read_excel(github_excel_url, engine = engine)
Equity['Security Code'] = Equity['Security Code'].astype(str)
Equity.head() # Get the list of stocks in Nifty50

,Security Code,Issuer Name,Security Id,Security Name,Status,Group,Face Value,ISIN No,Industry,Instrument,Sector Name,Industry New Name,Igroup Name,ISubgroup Name
0,500002,ABB India Limited,ABB,ABB India Limited,Active,A,2.0,INE117A01022,Heavy Electrical Equipment,Equity,Industrials,Capital Goods,Electrical Equipment,Heavy Electrical Equipment
1,500003,Aegis Logistics Ltd.,AEGISLOG,AEGIS LOGISTICS LTD.,Active,A,1.0,INE208C01025,Trading - Gas,Equity,Energy,"Oil, Gas & Consumable Fuels",Gas,Trading - Gas
2,500008,Amara Raja Energy & Mobility Limited,ARE&M,Amara Raja Energy & Mobility Ltd,Active,A,1.0,INE885A01032,Auto Components & Equipments,Equity,Consumer Discretionary,Automobile and Auto Components,Auto Components,Auto Components & Equipments
3,500009,"Ambalal Sarabhai Enterprise Ltd.,",AMBALALSA,AMBALAL SARABHAI ENTERPRISES LTD.,Active,XT,10.0,INE432A01017,Pharmaceuticals,Equity,Healthcare,Healthcare,Pharmaceuticals & Biotechnology,Pharmaceuticals
4,500012,"Andhra Petrochemicals Ltd.,",ANDHRAPET,ANDHRA PETROCHEMICALS LTD.,Active,X,10.0,INE714B01016,Commodity Chemicals,Equity,Commodities,Chemicals,Chemicals & Petrochemicals,Commodity Chemicals


In [0]:

nifty50_stock_symbols = [ "ADANIENT", "ADANIPORTS", "APOLLOHOSP", "ASIANPAINT", "AXISBANK",
    "BAJAJ-AUTO", "BAJFINANCE", "BAJAJFINSV", "BPCL", "BHARTIARTL",
    "BRITANNIA", "CIPLA", "COALINDIA", "DIVISLAB", "DRREDDY", "EICHERMOT",
    "GRASIM", "HCLTECH", "HDFCBANK", "HDFCLIFE", "HEROMOTOCO", "HINDALCO",
    "HINDUNILVR", "ICICIBANK", "ITC", "INDUSINDBK", "INFY", "JSWSTEEL",
    "KOTAKBANK", "LTIM", "LT", "M&M", "MARUTI", "NTPC", "NESTLEIND",
    "ONGC", "POWERGRID", "RELIANCE", "SBILIFE", "SBIN", "SUNPHARMA",
    "TCS", "TATACONSUM", "TATAMOTORS", "TATASTEEL", "TECHM", "TITAN",
    "UPL", "ULTRACEMCO", "WIPRO"
]

nifty50_SqDF= Equity[Equity['Security Id'].isin(nifty50_stock_symbols)].reset_index(drop=True)
nifty50_SqDF.rename(columns={'Group': 'CompanyGroup'}, inplace=True)
nifty50_SqDF.columns = nifty50_SqDF.columns.str.replace(' ', '')

b = BSE(update_codes=True)
result_dfs = []
sqcode_ListNifty50 = nifty50_SqDF['SecurityCode'].values
for sqCode in sqcode_ListNifty50:
    try:
        stock_data = b.getQuote(sqCode)
        stock_df = pd.DataFrame([stock_data])
        result_dfs.append(stock_df)
        time.sleep(0.5)
    except IndexError:
        print(f"IndexError for {sqCode}: Data not available")
        
nifty50_OverviewTable_SF = pd.concat(result_dfs, ignore_index=True)
nifty50DailyTable = pd.DataFrame()

for scripCode in nifty50_OverviewTable_SF['scripCode']:
    try:
        stock_data = b.getQuote(scripCode)
        stock_df = pd.DataFrame([stock_data])
        nifty50DailyTable = pd.concat([nifty50DailyTable, stock_df.iloc[:, :-2]], ignore_index=True)
        time.sleep(1.5)
    except IndexError:
        print(f"IndexError for {scripCode}: Data not available")
nifty50DailyTable.rename(columns={'group': 'sharegroup'}, inplace=True)
nifty50DailyTable.rename(columns={'52weekHigh': 'fiftytwoweekHigh'}, inplace=True)
nifty50DailyTable.rename(columns={'52weekLow': 'fiftytwoweekLow'}, inplace=True)
nifty50DailyTable.rename(columns={'2WeekAvgQuantity': 'twoWeekAvgQuantity'}, inplace=True)
nifty50DailyTable.head(n=2)

In [0]:
nifty50DailyTableTest_SF = nifty50DailyTable.copy()
# Convert 'updatedOn' column to datetime and extract date
nifty50DailyTableTest_SF['updatedOn'] = pd.to_datetime(nifty50DailyTableTest_SF['updatedOn'], format='%d %b %y | %I:%M %p', errors='coerce')

# Check if there are any invalid or missing date values
if pd.isna(nifty50DailyTableTest_SF['updatedOn']).any():
    print("There are invalid or missing date values in the 'updatedOn' column.")
else:
    # Extract date from 'updatedOn' column and convert the column to datetime
    nifty50DailyTableTest_SF['updatedOn'] = pd.to_datetime(nifty50DailyTableTest_SF['updatedOn'].dt.date)

if 'totalTradedValueCr' not in nifty50DailyTableTest_SF.columns:
   # Assuming nifty50DailyTableTest_SF is your DataFrame
    nifty50DailyTableTest_SF['totalTradedValueCr'] = pd.to_numeric(nifty50DailyTableTest_SF['totalTradedValue'].str.replace(',', '').str.replace(' Cr.', '', regex=True), errors='coerce')  # Convert to numeric and handle 'Cr.'
    nifty50DailyTableTest_SF['totalTradedQuantityLakh'] = pd.to_numeric(nifty50DailyTableTest_SF['totalTradedQuantity'].str.replace(',', '').str.replace(' Lakh', '', regex=True), errors='coerce')  # Convert to numeric and handle 'Lakh'
    nifty50DailyTableTest_SF['twoWeekAvgQuantityLakh'] = pd.to_numeric(nifty50DailyTableTest_SF['twoWeekAvgQuantity'].str.replace(',', '').str.replace(' Lakh', '', regex=True), errors='coerce')  # Convert to numeric and handle 'Lakh'
    nifty50DailyTableTest_SF['marketCapFullCr'] = pd.to_numeric(nifty50DailyTableTest_SF['marketCapFull'].str.replace(',', '').str.replace(' Cr.', '', regex=True), errors='coerce')  # Convert to numeric and handle 'Cr.'
    nifty50DailyTableTest_SF['marketCapFreeFloatCr'] = pd.to_numeric(nifty50DailyTableTest_SF['marketCapFreeFloat'].str.replace(',', '').str.replace(' Cr.', '', regex=True), errors='coerce')  # Convert to numeric and handle 'Cr.'

    # Drop original columns
    nifty50DailyTableTest_SF.drop(['totalTradedValue', 'totalTradedQuantity','twoWeekAvgQuantity', 'marketCapFull', 'marketCapFreeFloat'], axis=1, inplace=True)

nifty50DailyDailyData_spark_df = spark.createDataFrame(nifty50DailyTableTest_SF)
nifty50DailyTableTest_SF.head(n=2)

,companyName,currentValue,change,pChange,updatedOn,securityID,scripCode,sharegroup,faceValue,industry,previousClose,previousOpen,dayHigh,dayLow,fiftytwoweekHigh,fiftytwoweekLow,weightedAvgPrice,totalTradedValueCr,totalTradedQuantityLakh,twoWeekAvgQuantityLakh,marketCapFullCr,marketCapFreeFloatCr
0,Bajaj Finance Limited,7021.85,-50.40,-0.71,2023-11-24,BAJFINANCE,500034,A / S&P BSE SENSEX,2.00,Financial Services,7072.25,7100.00,7110.00,7012.85,8190.00,5487.25,7047.46,19.81,0.28,0.47,433966.53,190945.28
1,CIPLA LTD.,1198.85,30.25,2.59,2023-11-24,CIPLA,500087,A / S&P BSE 100,2.00,Healthcare,1168.60,1193.95,1205.25,1178.55,1283.00,852.00,1195.38,20.85,1.74,0.68,96786.21,63878.90


In [0]:
from pyspark.sql import SparkSession

# Create a Spark session
spark = SparkSession.builder.appName("Nifty50DailyData").getOrCreate()
jdbcHostname = "mainsqldbserver.database.windows.net"
jdbcDatabase = "nifty50db"
jdbcPort = 1433
jdbcUrl = "jdbc:sqlserver://{0}:{1};database={2}".format(jdbcHostname, jdbcPort, jdbcDatabase)
connectionProperties = {
  "user" : "jangid6",
  "password" : "Notmypassword",
  "driver" : "com.microsoft.sqlserver.jdbc.SQLServerDriver"
}
table_name = "nifty50Table"
nifty50_dailydata_exists = False
try:
  spark.read.jdbc(url=jdbcUrl, table=table_name, properties=connectionProperties)
  nifty50_dailydata_exists = True
except:
    columns = [
      "`companyName` VARCHAR(50)",
      "`currentValue` FLOAT",
      "`change` FLOAT",
      "`pChange` FLOAT",
      "`updatedOn` DATE",
      "`securityID` VARCHAR(50)",
      "`scripCode` VARCHAR(50)",
      "`sharegroup` VARCHAR(50)",
      "`faceValue` FLOAT",
      "`industry` VARCHAR(50)",
      "`previousClose` FLOAT",
      "`previousOpen` FLOAT",
      "`dayHigh` FLOAT",
      "`dayLow` FLOAT",
      "`fiftytwoweekHigh` FLOAT",
      "`fiftytwoweekLow` FLOAT",
      "`weightedAvgPrice` FLOAT",
      "`totalTradedQuantityLakh` FLOAT",
      "`totalTradedValueCr` FLOAT",
      "`twoWeekAvgQuantityLakh` FLOAT",
      "`marketCapFullCr` FLOAT",
      "`marketCapFreeFloatCr` FLOAT"
    ]
    drop_query = f"DROP TABLE {table_name}"
    spark.sql(drop_query)
    create_query = f"CREATE TABLE {table_name} ({','.join(columns)})"
    spark.sql(create_query)
    nifty50DailyDailyData_spark_df.createOrReplaceTempView("nifty50dailydata_temp_table")
    nifty50DailyDailyData_spark_df.write.jdbc(url=jdbcUrl, table=table_name, mode="overwrite",
                                            properties=connectionProperties)
    nifty50_dailydata_exists = True

In [0]:
queryMaxDate = f"SELECT MAX(updatedOn) as max_date FROM {table_name}"
queryresult = spark.read.jdbc(url=jdbcUrl, table="({0}) temp".format(queryMaxDate), properties=connectionProperties)
sql_max_date = queryresult.first()[0]
df_max_updatedOn = nifty50DailyTableTest_SF['updatedOn'].max()
if (sql_max_date == None) or (sql_max_date < df_max_updatedOn):
    nifty50DailyDailyData_spark_df.write.jdbc(url=jdbcUrl, table=table_name, mode="append",
                                            properties=connectionProperties)
    print("[update Completed] Table is updated with latest data")
else:
    print("[No Update Required] Table is already updated with latest data")

[No Update Required] Table is already updated with latest data
